# Variational Classifier

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [2]:
dev = qml.device("default.qubit", wires = 4)

In [3]:
def layer(W):
    qml.Rot(*W[0], wires = 0)
    qml.Rot(*W[1], wires = 1)
    qml.Rot(*W[2], wires = 2)
    qml.Rot(*W[3], wires = 3)
    
    qml.CNOT(wires = [0,1])
    qml.CNOT(wires = [1,2])
    qml.CNOT(wires = [2,3])
    qml.CNOT(wires = [3,0])

In [4]:
def statepreparation(x):
    qml.BasisState(x, wires = [0,1,2,3])

In [5]:
@qml.qnode(dev)
def circuit(weights, x):
    statepreparation(x)
    for W in weights:
        layer(W)
    
    return qml.expval(qml.PauliZ(0))

In [6]:
def Variational_classifier(var, x):
    weights = var[0]
    bias = var[1]
    return circuit(weights, x) + bias

In [ ]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)
    return loss

In [10]:
def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss /len(labels)
    return loss

In [11]:
def cost(var, X, Y):
    predictions = [variational_clssifier(var, x) for x in X]
    return square_loss(Y, predictions)

First we optimize the circuit for the simple parity dataset.

In [12]:
data = np.loadtxt("parity.txt")
X = np.array(data[:,:-1], requires_grad = False)
Y = np.array(data[:, -1], requires_grad = False)
Y = Y * 2 - np.ones(len(Y))

for i in range(len(X)):
    print('X = {}, Y = {: d}'.format(X[i], int(Y[i])))

X = [0. 0. 0. 0.], Y = -1
X = [0. 0. 0. 1.], Y =  1
X = [0. 0. 1. 0.], Y =  1
X = [0. 0. 1. 1.], Y = -1
X = [0. 1. 0. 0.], Y =  1
X = [0. 1. 0. 1.], Y = -1
X = [0. 1. 1. 0.], Y = -1
X = [0. 1. 1. 1.], Y =  1
X = [1. 0. 0. 0.], Y =  1
X = [1. 0. 0. 1.], Y = -1
X = [1. 0. 1. 0.], Y = -1
X = [1. 0. 1. 1.], Y =  1
X = [1. 1. 0. 0.], Y = -1
X = [1. 1. 0. 1.], Y =  1
X = [1. 1. 1. 0.], Y =  1
X = [1. 1. 1. 1.], Y = -1


In [14]:
np.random.seed(0)
num_qubits = 4
num_layers = 2
var_init = (0.01 * np.random.randn(num_layers, num_qubits, 3), 0.0)

In [15]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

In [ ]:
var = var_init
for it in range(25):
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    var = opt.step()